In [1]:
from tkinter import Tk, Button
from tkinter.font import Font
from copy import deepcopy
import gym
import numpy as np
import time

class QLearningAgent(object):
    # Inicializar Parametros
    def __init__(self, stateSpace_size, actionSpace_size, learning_rate=0.01, gamma=0.9, epsilon=0.1):
        self.stateSpace_size = stateSpace_size  #      
        self.actionSpace_size = actionSpace_size  #   Tama;o del Espacio
        self.learning_rate = learning_rate  #    
        self.gamma = gamma  # 
        self.epsilon = epsilon  # 
        self.QTable = np.zeros((stateSpace_size, actionSpace_size))  # Tabla Q
        self.runTimes = 10

   

    # Estable el numero de iteraciones
    def setRunTimes(self, run_times):
        self.runTimes = run_times

    # Configuraciones
    def setTrust(self, trust_ratio):
        self.epsilon = trust_ratio

    # Selecciona una estado de Accion
    def getAction(self, state):
        sand = np.random.uniform(0, 1)
        if sand <= self.epsilon:
            action = np.random.choice(self.actionSpace_size)
        else:
            action = self.chooseActionFromQTable(state)
        return action

    #Tabla Q      
    def chooseActionFromQTable(self, state):
        Q_max = np.max(self.QTable[state, :])
        action_list = np.where(self.QTable[state, :] == Q_max)[0]
        action = np.random.choice(action_list)
        return action

    # Actualizar tabla
    def updateQTable(self, state, action, reward, next_state, done):
        if done:
            Q_future = 0
        else:
            Q_future = np.max(self.QTable[next_state, :])
        updateValue = self.learning_rate * (reward + self.gamma * Q_future - self.QTable[state, action])
        self.QTable[state, action] += updateValue

    # Mostrar Tabla
    def showQTable(self):
        print(self.QTable)

    # Leer los datos de la tabla
    def loadQTable(self, path):
        self.QTable = np.load(path)
        print(path + ' loaded.')

    #Guarda los datos de la tabla
    def saveQTable(self, path):
        np.save(path, self.QTable)
        print(path + ' saved.')

    

    # Codigo
    def run(self, env, agent, render):
        total_steps = 0
        while True:
            state = env.reset()  #  Restablece el entorno, es como reiniciar el juego y empieza otra iteracion
            episode_reward = 0
            episode_steps = 0
            if total_steps >= self.runTimes:
                break
            while True:
                action = self.chooseActionFromQTable(state)  # Selecciona un algoritmo de la libreria
                next_state, reward, done, info = env.step(action)  # Interactua con el escenario
                state = next_state  # Guarda el estado anteriro
                episode_reward += -reward
                episode_steps += 1  # Guarda los pasos que va a seguir el episodio
                if render:
                    time.sleep(0.5)
                    env.render()  # Renderiza un nuevo grafico
                if done:
                    total_steps += 1
                    print("Iteracion " + str(total_steps) + ", numero de pasos " + str(episode_steps) + ", premio " + str(
                        episode_reward))
                    break



render = False # valor para mostrar el entorno
env = gym.make('Taxi-v3')# se puede cambiar por FrozenLake-v0, FrozenLake8x8-v0 algoritmos de la pagian gym

agent = QLearningAgent(
        stateSpace_size=env.observation_space.n,
        actionSpace_size=env.action_space.n,
        learning_rate=0.1,
        gamma=0.9,
        epsilon=0.3)  # Define un agente

def jugar():
    agent.showQTable()
    agent.setRunTimes(1)#numero de episodios
    agent.run(env, agent, render)  




In [2]:
from neo4j import GraphDatabase
class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
    def close(self):
        self._driver.close()
    
    def crear_nodo(self, tx, nombre,v1,v2,v3):
        tx.run("MERGE (jugador:Lugar {name:$nombre, ciudad:$v1 })"    
        "SET jugador.embedding = [$v2, $v3]",nombre=nombre
        ,v1=v1,v2=v2,v3=v3)
        
    def recomendacion(self,tx):
        result = tx.run("MATCH (m:Lugar)\n"
                        "WITH {item:id(m), weights: m.embedding} AS userData\n"
                        "WITH collect(userData) AS data\n"
                        "CALL gds.alpha.similarity.pearson.stream({\n"
                        "data: data,\n"
                        "skipValue: null\n"
                        "})\n"
                        "YIELD item1, item2, similarity\n"
                        "RETURN gds.util.asNode(item1).name AS from, gds.util.asNode(item2).name AS to, similarity\n"
                        "ORDER BY similarity DESC")
        for record in result:
            r1=(record["from"])
            r2=(record["to"])
            r3=(record["similarity"])
            if r1 ==nombre.get() and r3>=0.80:
                resultado.insert(tk.END, "\n"+r2)

In [6]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import tkinter as tk

raiz1 = Tk()

def clearTextInput():
    resultado.delete("1.0","end")


def guardar():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
    with neo4j._driver.session() as session:
        session.write_transaction(neo4j.crear_nodo , nombre.get(), ciudad.get(), float(longitud.get()), float(latitud.get()))
        
def buscar():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
    with neo4j._driver.session() as session:
        session.read_transaction(neo4j.recomendacion)
        

    
raiz1.geometry('400x500')  # anchura x altura

raiz1.title('Examen IA')
Label(raiz1, text="Examen IA").place(x=155, y=0)




Label(raiz1, text="Nombre del Jugador").place(x=55, y=100)
nombre = StringVar()
txtnombe=Entry(raiz1,textvariable=nombre).place(x=300,y=100)

Label(raiz1, text="Ciudad Natal.").place(x=55, y=25)
ciudad = StringVar()
txtciudad=Entry(raiz1,textvariable=ciudad).place(x=300,y=25)


Label(raiz1, text="Latitud.").place(x=55, y=50)
latitud = DoubleVar()
txtlatitud=Entry(raiz1,textvariable=latitud).place(x=300,y=50)


Label(raiz1, text="Longitud.").place(x=55, y=75)
longitud = DoubleVar()
txtlongitud=Entry(raiz1,textvariable=longitud).place(x=300,y=75)

ttk.Button(raiz1, text='Guardar Datos', command=guardar).place(x=50, y=250)
ttk.Button(raiz1, text='Recomendar', command=buscar).place(x=150, y=250)
ttk.Button(raiz1, text='Juego', command=jugar).place(x=250, y=250)

Label(raiz1, text="Recomendacion").place(x=55, y=300)  
    
resultado = Text(raiz1)
resultado.place(x = 55, y=325, width=300, height=100)

raiz1.mainloop()

